In [3]:
%pip install alpha-vantage

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# GET DATA FROM API

In [6]:
API_KEY = 'N5E4Q2IN4MMPFWGY'

In [7]:
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key=API_KEY, output_format='pandas')
data = ts.get_intraday(symbol='MSFT',interval='15min', outputsize='full')
data = data[0]
data.rename(lambda x: x[2:].strip(), axis='columns', inplace=True)
data

,open,high,low,close,volume
date,,,,,
2023-08-08 19:45:00,325.85,326.03,325.61,325.66,7100.0
2023-08-08 19:30:00,325.95,326.04,325.62,325.62,5825.0
2023-08-08 19:15:00,326.00,326.05,325.88,325.88,2784.0
2023-08-08 19:00:00,325.96,326.05,325.87,326.05,5284.0
2023-08-08 18:45:00,325.86,326.05,325.86,325.96,502.0
...,...,...,...,...,...
2023-07-11 05:00:00,331.77,331.80,331.50,331.69,1734.0
2023-07-11 04:45:00,331.45,331.82,331.20,331.77,771.0
2023-07-11 04:30:00,331.82,331.82,331.20,331.42,2210.0


## Technical indicator for analyze data

In [8]:

from alpha_vantage.techindicators import TechIndicators
ti = TechIndicators(key=API_KEY, output_format='pandas')

### Rate of Change (ROC)

In [9]:
indicator_roc = ti.get_roc(symbol='MSFT', interval='15min', time_period=20)
indicator_roc = indicator_roc[0]
indicator_roc

,ROC
date,
2023-07-11 09:00:00,0.0392
2023-07-11 09:15:00,-0.2562
2023-07-11 09:30:00,-0.4840
2023-07-11 09:45:00,-0.9208
2023-07-11 10:00:00,-1.3024
...,...
2023-08-08 18:45:00,0.4778
2023-08-08 19:00:00,0.4560
2023-08-08 19:15:00,0.3418


### Simple Moving Average (SMA)

In [10]:
indicator_sma = ti.get_sma(symbol='MSFT', interval='15min', time_period=20)
indicator_sma = indicator_sma[0]
indicator_sma

,SMA
date,
2023-07-11 08:45:00,332.0935
2023-07-11 09:00:00,332.1000
2023-07-11 09:15:00,332.0575
2023-07-11 09:30:00,331.9773
2023-07-11 09:45:00,331.8246
...,...
2023-08-08 18:45:00,325.6097
2023-08-08 19:00:00,325.6837
2023-08-08 19:15:00,325.7392


### Relative Strength Index (RSI)

In [11]:
indicator_rsi = ti.get_rsi(symbol='MSFT', interval='15min', time_period=20)
indicator_rsi = indicator_rsi[0]
indicator_rsi

,RSI
date,
2023-07-11 09:00:00,51.4739
2023-07-11 09:15:00,40.0874
2023-07-11 09:30:00,32.8602
2023-07-11 09:45:00,27.7092
2023-07-11 10:00:00,23.0609
...,...
2023-08-08 18:45:00,51.0791
2023-08-08 19:00:00,51.9842
2023-08-08 19:15:00,50.1398


### Bolling Bands (BBANDS)

In [12]:
def BollingerBand(data, n_loockback, n_std=2):
    hlc_avg = (data.high + data.low + data.close)/3
    data_std = hlc_avg.rolling(window=n_loockback).std()
    data_upper = data.SMA + n_std * data_std
    data_lower = data.SMA - n_std * data_std
    return data.SMA, data_lower, data_upper

#### Merge indicators to data

In [14]:
import pandas as pd

In [16]:
original_df = pd.merge(data, indicator_roc, on='date', how='inner')
original_df = pd.merge(original_df, indicator_sma, on='date', how='inner')
original_df = pd.merge(original_df, indicator_rsi, on='date', how='inner')
# original_df = pd.merge(original_df, indicator_bb, on='date', how='inner')
original_df["Real Middle Band"], original_df["Real Lower Band"], original_df["Real Upper Band"] = BollingerBand(original_df, 20, 2)
original_df = original_df.dropna()
original_df = original_df.iloc[::-1]
original_df.to_csv('./DATA/MSFT.csv')
original_df

,open,high,low,close,volume,ROC,SMA,RSI,Real Middle Band,Real Lower Band,Real Upper Band
date,,,,,,,,,,,
2023-07-11 09:00:00,332.16,332.300,331.82,332.130,31529.0,0.0392,332.1000,51.4739,332.1000,329.782799,334.417201
2023-07-11 09:15:00,332.13,332.190,330.11,330.940,68332.0,-0.2562,332.0575,40.0874,332.0575,329.817853,334.297147
2023-07-11 09:30:00,331.06,331.450,329.25,329.816,2365216.0,-0.4840,331.9773,32.8602,331.9773,329.750536,334.204064
2023-07-11 09:45:00,329.79,329.910,328.28,328.715,1514042.0,-0.9208,331.8246,27.7092,331.8246,329.594368,334.054832
2023-07-11 10:00:00,328.72,328.755,327.00,327.370,1678788.0,-1.3024,331.6086,23.0609,331.6086,329.432947,333.784253
...,...,...,...,...,...,...,...,...,...,...,...
2023-08-08 14:00:00,324.43,324.780,323.99,324.570,398393.0,-0.8826,324.6160,40.0222,324.6160,321.290507,327.941493
2023-08-08 14:15:00,324.57,325.260,324.57,324.770,487948.0,-0.6668,324.5070,41.6491,324.5070,321.204619,327.809381
2023-08-08 14:30:00,324.79,324.910,324.25,324.313,414302.0,-0.9671,324.3487,39.0982,324.3487,321.054359,327.643041
